In [47]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

Main.DeepSpace

In [48]:
using DataFrames, DataFramesMeta
using Statistics
using ScikitLearn
using Main

@sk_import linear_model: LinearRegression
@sk_import linear_model: ElasticNet

PyObject <class 'sklearn.linear_model.coordinate_descent.ElasticNet'>

In [49]:
KEY = "2019vagle"
YEAR = "2019"

"2019"

In [50]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)

(["adjustPoints", "autoPoints", "bay1", "bay2", "bay3", "bay4", "bay5", "bay6", "bay7", "bay8"  …  "techFoulCount", "teleopPoints", "topLeftRocketFar", "topLeftRocketNear", "topRightRocketFar", "topRightRocketNear", "totalPoints", "key", "level", "event"], 180×54 DataFrame. Omitted printing of 50 columns
│ Row │ adjustPoints │ autoPoints │ bay1          │ bay2          │
│     │ Int64⍰       │ Int64⍰     │ String⍰       │ String⍰       │
├─────┼──────────────┼────────────┼───────────────┼───────────────┤
│ 1   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 2   │ 0            │ 15         │ Panel         │ Panel         │
│ 3   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 4   │ 0            │ 15         │ Panel         │ PanelAndCargo │
│ 5   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 6   │ 0            │ 15         │ PanelAndCargo │ PanelAndCargo │
│ 7   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 8   │ 0     

In [51]:
DeepSpace.calculated_columns!(scores)
scores[:ownPoints] = scores.totalPoints .- scores.foulPoints .- (3 .* scores.foulCount) .- (9 * scores.techFoulCount)

180-element Array{Int64,1}:
 83
 34
 81
 38
 70
 32
 62
 46
 51
 54
 65
 64
 66
  ⋮
 20
 12
 70
 45
 59
 47
 57
 60
 74
 66
 51
 55

In [52]:
scores_train = @linq scores |> where(:level .== "qm")
teams_train = @linq teams |> where(:level .== "qm")

scores_test = @linq scores |> where(:level .!= "qm")
teams_test = @linq teams |> where(:level .!= "qm")

(ß, teamnums, teamindex) = Lib.opr_ß(scores_train, teams_train ; shortevent = true)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 1.0 0.0 … 0.0 0.0], Union{Missing, String}["frc1080", "frc1086", "frc122", "frc1262", "frc1413", "frc1522", "frc1598", "frc1599", "frc1610", "frc1793"  …  "frc6021", "frc617", "frc6189", "frc619", "frc6334", "frc6802", "frc6882", "frc7330", "frc7429", "frc977"], Dict("frc1086"=>2,"frc3359"=>16,"frc617"=>29,"frc540"=>23,"frc1522"=>6,"frc3361"=>17,"frc3258"=>15,"frc5546"=>24,"frc619"=>31,"frc1793"=>10…))

In [53]:
model = LinearRegression(fit_intercept=false)
components = DataFrame([teamnums], [:team])

for col in Lib.numeric_col_names(scores_train)
    y = convert.(Float64, coalesce(scores_train[col], -1.0))
    modelfit = ScikitLearn.fit!(model, ß, y)
    components[col] = modelfit[:coef_]
end
components

,team,adjustPoints,autoPoints,cargoPoints,completeRocketRankingPoint,completedRocketFar,completedRocketNear,foulCount,foulPoints,habClimbPoints,habDockingRankingPoint,hatchPanelPoints,rp,sandStormBonusPoints,techFoulCount,teleopPoints,totalPoints,rocketLowPoints,rocketMidPoints,rocketTopPoints,rocketPoints,ownPoints
,String⍰,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,frc1080,0.0,3.02345,4.04895,-0.014973,-0.0159382,0.0,-0.156126,-0.369823,3.35403,0.0469256,0.663347,0.199775,3.02345,-0.0150174,8.06633,10.72,1.06021,0.745208,0.590239,2.39566,11.6933
2,frc1086,0.0,3.42255,4.54003,-0.00639716,0.00695878,0.0,-0.0874686,0.172497,3.32256,-0.0209212,0.711476,-0.0585473,3.42255,-0.0219442,8.57406,12.1691,-0.327525,-0.449393,-0.229599,-1.00652,12.4565
3,frc122,0.0,3.99446,5.02112,-0.0197229,-0.0140204,0.0,-0.0322353,0.13493,4.00184,0.0275384,1.99484,0.293968,3.99446,0.0427331,11.0178,15.1472,0.34146,-0.135938,-0.161916,0.0436063,14.7244
4,frc1262,0.0,3.36678,8.67643,0.0894812,0.0850349,0.0,0.259097,0.0798699,5.01688,0.108632,6.14141,1.06552,3.36678,-0.0358919,19.8347,23.2814,5.67182,3.46605,2.63103,11.7689,22.7472
5,frc1413,0.0,3.09664,6.29119,-0.000688515,-0.00246549,0.0,-0.202185,1.81764,5.05944,0.175867,1.77831,0.778047,3.09664,-0.0262021,13.1289,18.0432,1.51251,0.977214,-0.0293405,2.46038,17.068
6,frc1522,0.0,1.73207,3.01897,0.00622271,0.00321153,0.0,0.0347033,1.90515,-0.0535299,-0.13141,0.980086,0.206705,1.73207,-0.0282698,3.94552,7.58274,-0.105731,-0.301557,-0.134337,-0.541625,5.82791
7,frc1598,0.0,3.83297,1.51,-0.01262,-0.00749037,0.0,-0.328096,0.0161585,0.879405,-0.0213349,0.515678,-0.180724,3.83297,-0.0491174,2.90509,6.75422,-0.844455,0.371346,0.0269018,-0.446208,8.1644
8,frc1599,0.0,3.62871,4.21255,-0.0220319,-0.00786757,0.0,0.396397,-0.387009,8.11946,0.347738,2.63353,0.943197,3.62871,0.0667287,14.9656,18.2073,0.776863,-0.153697,0.506748,1.12991,16.8045
9,frc1610,0.0,5.50004,1.41653,0.0810915,0.0855671,0.0,0.253725,0.977626,2.22768,-0.0622859,10.5294,1.12774,5.50004,0.0613888,14.1736,20.6512,4.3744,3.41102,3.02588,10.8113,18.3599


In [54]:
colname = [:rocketTopPoints, :rocketMidPoints, :rocketLowPoints, :cargoPoints, :team]
sort(components, colname, rev=true)[colname]

,rocketTopPoints,rocketMidPoints,rocketLowPoints,cargoPoints,team
,Float64,Float64,Float64,Float64,String⍰
1,3.02588,3.41102,4.3744,1.41653,frc1610
2,2.63103,3.46605,5.67182,8.67643,frc1262
3,1.19204,1.4479,3.13713,-1.3132,frc6334
4,0.657643,0.843531,1.73745,6.36575,frc540
5,0.590239,0.745208,1.06021,4.04895,frc1080
6,0.506748,-0.153697,0.776863,4.21255,frc1599
7,0.491331,-0.360258,3.48556,2.88719,frc1793
8,0.395402,1.02737,1.06594,2.06467,frc2106
9,0.388867,-0.233013,-0.873532,2.06523,frc539


In [56]:
using CSV
CSV.write("../output/$(KEY)_components.csv", components)

"../output/2019vagle_components.csv"